### Jigsaw Unintended Bias in Toxicity Classification

1. [Introduction](#Introduction) <br>
2. [Functions](#Functions) <br>
3. [Data](#Data) <br>
4. [Embedding](#Embedding) <br>
5. [Model](#Model) <br>
6. [Submission](#Submission) <br>




#### Introduction

This is a simple Neural Network (NN) Kernel using Keras preprocessing features and TensorFlow for backend processing.  This is a simple baseline kernel that does not perform as well as the more sophisticated kernels but does include several concise loading functions.  This kernel is intended to be used as a starting point for more elaborate kernels.    

In [1]:
import os 

import numpy as np

import pandas as pd

import tensorflow as tf

from tensorflow import keras

from keras.preprocessing import text, sequence

from keras.preprocessing.text import Tokenizer

raw_train = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/train.csv')

raw_test = pd.read_csv('../input/jigsaw-unintended-bias-in-toxicity-classification/test.csv')

vocab_size = 100000

max_length = 220

text_column = 'comment_text'

target_column = 'target'

char_filter = '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'

embedding_loc = '../input/fasttext-crawl-300d-2m/crawl-300d-2M.vec'

Using TensorFlow backend.


#### Functions

The prepare_data function takes the pd.DataFrame training and test data objects and returns tokenized and padded x data, y training data and the fitted keras.Tokenizer object.  The build_embedding function creates the embedding index, embedding matrix and embedding layer from a word2vec source and a word index dictionary.  The build_embedding function was motivated by Dieter's excellent <a href='https://www.kaggle.com/christofhenkel/keras-baseline-lstm-attention-5-fold'> baseline lstm + attention 5-fold kernel </a>.    

In [2]:
def prepare_data(train_data, test_data, text_column, target_column, char_filter, vocab_size, max_length):
    
    raw_x_test = test_data[text_column].astype(str)
    
    raw_x_train = train_data[text_column].astype(str)
    
    y_train = train_data[target_column].values
    
    tokenizer = Tokenizer(num_words=vocab_size, filters=char_filter)

    tokenizer.fit_on_texts(list(raw_x_train) + list(raw_x_test))
    
    x_test = tokenizer.texts_to_sequences(raw_x_test)
    
    x_train = tokenizer.texts_to_sequences(raw_x_train)
    
    x_test = sequence.pad_sequences(x_test, maxlen = max_length)
    
    x_train = sequence.pad_sequences(x_train, maxlen = max_length)
    
    return x_train, y_train, x_test, tokenizer

def vec_parser(word, *coeffs):
    
    return word, np.asarray(coeffs, dtype='float32')

def build_embedding(embedding_loc, word_index, max_length, dimensionality):
    
    embedding_index = dict(vec_parser(*line.strip().split(" ")) for line in open(embedding_loc, encoding='utf-8'))
    
    embedding_matrix = np.zeros((len(word_index) + 1, dimensionality))
    
    for word, i in word_index.items():
        
        try:
            
            embedding_matrix[i] = embedding_index[word]
        
        except:
            
            embedding_matrix[i] = embedding_index["unknown"]
            
    embedding_layer = keras.layers.Embedding(len(word_index)+1, 
                                         dimensionality, 
                                         weights=[embedding_matrix], 
                                         input_length=max_length, 
                                         trainable=False)
    
    return embedding_index, embedding_matrix, embedding_layer

def joint_shuffle(x_data, y_data):
    
    if len(x_data) == len(y_data):
    
        p = np.random.permutation(len(x_data))
    
    return x_data[p], y_data[p]

#### Data

The data is loaded using the prepare_data function and the embedding is created using the build_embedding function.  

In [3]:
x_train, y_train, x_test, tokenizer = prepare_data(raw_train, 
                                        raw_test, 
                                        text_column, 
                                        target_column, 
                                        char_filter, 
                                        vocab_size, 
                                        max_length)

embedding_index, embedding_matrix, embedding_layer = build_embedding(embedding_loc, 
                                                                     tokenizer.word_index, 
                                                                     max_length, 
                                                                     300)


#### Model

The model used is a simple feedforward neural network that is very is essentially the same as the one proposed in the <a href= 'https://www.tensorflow.org/tutorials/keras/basic_text_classification'> Tensor Flow tutorial </a>.  

In [4]:
model = keras.Sequential()

model.add(embedding_layer)
model.add(keras.layers.GlobalAveragePooling1D())
model.add(keras.layers.Dense(16, activation=tf.nn.relu))
model.add(keras.layers.Dense(1, activation=tf.nn.sigmoid))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

model.summary()

x_train, y_train = joint_shuffle(x_train, y_train)

x_val = x_train[:10000]
partial_x_train = x_train[10000:]

y_val = y_train[:10000]
partial_y_train = y_train[10000:]

simple_callback = keras.callbacks.EarlyStopping(monitor='val_acc', 
                                                min_delta=0.005,
                                                patience=5,
                                                mode='max')

history = model.fit(partial_x_train,
                    partial_y_train,
                    epochs=1,
                    batch_size=512,
                    validation_data=(x_val, y_val),
                    verbose=1,
                    callbacks=[simple_callback])

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 220, 300)          98517000  
_________________________________________________________________
global_average_pooling1d (Gl (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 16)                4816      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 98,521,833
Trainable params: 4,833
Non-trainable params: 98,517,000
_________________________________________________________________
Train on 1794874 samples, validate on 10000 samples
Instructions for updating:
Use tf.cast instead.
1794874/1794874 [==============================] - 224s 125us/sample - loss: 0.3172 - 

#### Submission
The predicted classification is placed into a pd.DataFrame object for submission and saved locally.  

In [5]:
raw_results = model.predict(x_test)

results = np.average(raw_results, axis=1)

submission = pd.DataFrame.from_dict({
    'id': raw_test.id,
    'prediction': results})

submission.to_csv('submission.csv', index=False)